In [ ]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

def pg_con(): 
    con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'
    )
    return(con) 

In [1]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'    
)


In [ ]:
pg_con()

In [ ]:
c=pg_con().cursor() 

In [2]:
con

<connection object at 0x10c89a828; dsn: 'user=admin password=xxx dbname=postgres host=localhost port=30960', closed: 0>

In [3]:
cur=con.cursor()
cur

<cursor object at 0x10c507dd8; closed: 0>

In [ ]:
try:
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")
except:
    c.execute("rollback")
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")

In [ ]:
c.execute("INSERT INTO test2 VALUES(1,'abc')")

In [ ]:
c.execute("select * from test2")
c.fetchall()

In [4]:
import pandas as pd
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/Boston_Crime_Dataset.csv',encoding="latin_1",dtype='unicode' )  

In [5]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
OCCURRED_ON_DATE       object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
Lat                    object
Long                   object
Location               object
dtype: object

In [6]:
df_OFFENSE = df[['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION']].dropna()
df_OFFENSE['OFFENSE_DETAILS']=df_OFFENSE['OFFENSE_CODE']+'_'+df_OFFENSE['OFFENSE_CODE_GROUP']+'_'+df_OFFENSE['OFFENSE_DESCRIPTION']
df_OFFENSE['OFFENSE_DETAILS'] = df_OFFENSE['OFFENSE_DETAILS'].str.replace(' ', '')

In [7]:
df_OFFENSE.nunique()

OFFENSE_CODE           222
OFFENSE_CODE_GROUP      97
OFFENSE_DESCRIPTION    262
OFFENSE_DETAILS        302
dtype: int64

In [8]:
df_OFFENSE = df_OFFENSE.drop_duplicates(subset='OFFENSE_DETAILS', keep='first') #.reset_index()
df_OFFENSE['OFFENSE_ID']=range(1, len(df_OFFENSE.index)+1)
df_OFFENSE = df_OFFENSE[['OFFENSE_ID','OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION','OFFENSE_DETAILS']]
df_OFFENSE=df_OFFENSE.astype('str')

In [9]:
df_OFFENSE.isnull().sum()

OFFENSE_ID             0
OFFENSE_CODE           0
OFFENSE_CODE_GROUP     0
OFFENSE_DESCRIPTION    0
OFFENSE_DETAILS        0
dtype: int64

In [10]:
df_OFFENSE.head()

,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,OFFENSE_DETAILS
0,1,2403,Disorderly Conduct,DISTURBING THE PEACE,2403_DisorderlyConduct_DISTURBINGTHEPEACE
1,2,3201,Property Lost,PROPERTY - LOST,3201_PropertyLost_PROPERTY-LOST
2,3,2647,Other,THREATS TO DO BODILY HARM,2647_Other_THREATSTODOBODILYHARM
3,4,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,413_AggravatedAssault_ASSAULT-AGGRAVATED-BATTERY
4,5,3122,Aircraft,AIRCRAFT INCIDENTS,3122_Aircraft_AIRCRAFTINCIDENTS


In [11]:
cur.execute("DROP TABLE IF EXISTS OFFENSE");

In [12]:
cur.execute("""CREATE TABLE OFFENSE
(OFFENSE_ID int NOT NULL PRIMARY KEY,
OFFENSE_CODE integer NULL,
OFFENSE_CODE_GROUP text NULL,
OFFENSE_DESCRIPTION text NULL,
OFFENSE_DETAILS text NULL);""")

In [13]:
Offense_Cols = ",".join([str(i) for i in df_OFFENSE.columns.tolist()])

In [14]:
Offense_Cols

'OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,OFFENSE_DETAILS'

In [15]:
for i, row in df_OFFENSE.iterrows():
        sql = "INSERT INTO OFFENSE (" +Offense_Cols+ ") VALUES (" + "%s,"*(len(row)-1) + "%s);"
        cur.execute(sql,tuple(row))
        con.commit()

In [16]:
cur.execute("SELECT * FROM OFFENSE")
cur.fetchall()

[(1,
  2403,
  'Disorderly Conduct',
  'DISTURBING THE PEACE',
  '2403_DisorderlyConduct_DISTURBINGTHEPEACE'),
 (2,
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  '3201_PropertyLost_PROPERTY-LOST'),
 (3,
  2647,
  'Other',
  'THREATS TO DO BODILY HARM',
  '2647_Other_THREATSTODOBODILYHARM'),
 (4,
  413,
  'Aggravated Assault',
  'ASSAULT - AGGRAVATED - BATTERY',
  '413_AggravatedAssault_ASSAULT-AGGRAVATED-BATTERY'),
 (5,
  3122,
  'Aircraft',
  'AIRCRAFT INCIDENTS',
  '3122_Aircraft_AIRCRAFTINCIDENTS'),
 (6, 1402, 'Vandalism', 'VANDALISM', '1402_Vandalism_VANDALISM'),
 (7,
  3803,
  'Motor Vehicle Accident Response',
  'M/V ACCIDENT - PERSONAL INJURY',
  '3803_MotorVehicleAccidentResponse_M/VACCIDENT-PERSONALINJURY'),
 (8,
  3301,
  'Verbal Disputes',
  'VERBAL DISPUTE',
  '3301_VerbalDisputes_VERBALDISPUTE'),
 (9,
  802,
  'Simple Assault',
  'ASSAULT SIMPLE - BATTERY',
  '802_SimpleAssault_ASSAULTSIMPLE-BATTERY'),
 (10, 3410, 'Towed', 'TOWED MOTOR VEHICLE', '3410_Towed_TOWEDMOTORV

In [17]:
cur.execute("SELECT * FROM OFFENSE WHERE OFFENSE_DESCRIPTION ILIKE 'pro%'")
cur.fetchall()

[(2,
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  '3201_PropertyLost_PROPERTY-LOST'),
 (34,
  3207,
  'Property Found',
  'PROPERTY - FOUND',
  '3207_PropertyFound_PROPERTY-FOUND'),
 (40,
  3208,
  'Property Lost',
  'PROPERTY - MISSING',
  '3208_PropertyLost_PROPERTY-MISSING'),
 (53,
  3201,
  'property lost',
  'PROPERTY - LOST',
  '3201_propertylost_PROPERTY-LOST'),
 (71,
  1304,
  'Recovered Stolen Property',
  'PROPERTY - STOLEN THEN RECOVERED',
  '1304_RecoveredStolenProperty_PROPERTY-STOLENTHENRECOVERED'),
 (90,
  3106,
  'Property Related Damage',
  'PROPERTY - ACCIDENTAL DAMAGE',
  '3106_PropertyRelatedDamage_PROPERTY-ACCIDENTALDAMAGE'),
 (99,
  3202,
  'Property Found',
  'PROPERTY - LOST THEN LOCATED',
  '3202_PropertyFound_PROPERTY-LOSTTHENLOCATED'),
 (147,
  2631,
  'Other',
  'PROPERTY - CONCEALING LEASED',
  '2631_Other_PROPERTY-CONCEALINGLEASED'),
 (148,
  1605,
  'Prostitution',
  'PROSTITUTION - COMMON NIGHTWALKER',
  '1605_Prostitution_PROSTITUTION-COMMONNIGHTWA

In [18]:
cur.execute("DROP TABLE IF EXISTS INCIDENT")

In [19]:
cur.execute("""CREATE TABLE INCIDENT(
    INCIDENT_NUMBER text,
    OFFENSE_CODE integer,
    OFFENSE_CODE_GROUP text,
    OFFENSE_DESCRIPTION text,
    DISTRICT text, 
    REPORTING_AREA integer,
    OCCURRED_ON_DATE DATERANGE,
    YEAR integer,
    MONTH integer,
    DAY_OF_WEEK integer,
    HOUR integer,
    UCR_PART integer,
    STREET text,
    Lat float,
    Long float,
    Location text);""")
con.commit()

In [20]:
Incident_Cols = ",".join([str(i) for i in df.columns.tolist()])
Incident_Cols

'INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location'

In [21]:
for i, row in df.iterrows():
        sql = "INSERT INTO INCIDENT (" +Incident_Cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s);"
        cur.execute(sql,tuple(row))
        con.commit()

InvalidTextRepresentation: malformed range literal: "10.03.18 20:13"
LINE 1: ...derly Conduct','DISTURBING THE PEACE','E18','495','10.03.18 ...
                                                             ^
DETAIL:  Missing left parenthesis or bracket.
